In [1]:
test_file = "10_cols_1_mb"
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LargeRowBenchmark").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/04 12:39:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet(f"data/openai/parquet/{test_file}.parquet")
df.createOrReplaceTempView("T")
df.printSchema()
df.show(3)

root
 |-- id: long (nullable = true)
 |-- json_0: struct (nullable = true)
 |    |-- ended: boolean (nullable = true)
 |    |-- length: long (nullable = true)
 |    |-- text: string (nullable = true)
 |-- json_1: struct (nullable = true)
 |    |-- ended: boolean (nullable = true)
 |    |-- length: long (nullable = true)
 |    |-- text: string (nullable = true)
 |-- json_2: struct (nullable = true)
 |    |-- ended: boolean (nullable = true)
 |    |-- length: long (nullable = true)
 |    |-- text: string (nullable = true)
 |-- json_3: struct (nullable = true)
 |    |-- ended: boolean (nullable = true)
 |    |-- length: long (nullable = true)
 |    |-- text: string (nullable = true)
 |-- json_4: struct (nullable = true)
 |    |-- ended: boolean (nullable = true)
 |    |-- length: long (nullable = true)
 |    |-- text: string (nullable = true)
 |-- json_5: struct (nullable = true)
 |    |-- ended: boolean (nullable = true)
 |    |-- length: long (nullable = true)
 |    |-- text: string (nu

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|              json_0|              json_1|              json_2|              json_3|              json_4|              json_5|              json_6|              json_7|              json_8|              json_9|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|255000|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|{true, 134, Is th...|
|255001|{true, 713, Clint...|{true, 713, Clint...|{true, 713, Clint...|{true, 713, Clint...|{true, 713, Clint...|{true, 713,

In [3]:
# Q1. get the length of structure's long string
spark.sql("SELECT length(json_0['text']) FROM T").collect()

[Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576),
 Row(length(json_0.text)=1048576)]

In [4]:
# Q2. find word in structure's long string
spark.sql("SELECT id, json_0 FROM T WHERE contains(json_0['text'], 'House')").show()

+------+--------------------+
|    id|              json_0|
+------+--------------------+
|255002|{true, 173, House...|
+------+--------------------+



In [5]:
# Q3. lowercase the structure's long string
spark.sql("SELECT lower(json_0['text']) FROM T").show()

+--------------------+
|  lower(json_0.text)|
+--------------------+
|is this restauran...|
|clinton talks abo...|
|house majority wh...|
|insight course: l...|
|by jennie mcnulty...|
|the buddha's teac...|
|as part of a broa...|
|the atlanta falco...|
|front page torren...|
|they have changed...|
+--------------------+



In [6]:
# Q4. UDF: count word frequency in structure's long string
def word_count(text):
    word_count = {}
    for word in text.split():
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count


spark.udf.register("word_count", word_count)
spark.sql("SELECT word_count(json_0['text']) FROM T").collect()

[Row(word_count(json_0.text)='{specialty=2048, be=2048, hidden=1024, Could=2048, market?=2048, considered=2048, Yes=17408, for=4096, Is=8193, UnsureIs=1024, restaurant?=2048, good=4096, primarily=1024, gem=1024, dinner?=2048, waiIs=1023, mid-range=2048, and=2048, waiters=2048, have=2048, prices=2048, wai=1, moderate?=2048, Unsure=16384, a=6144, No=17408, path?=1024, or=1024, bakery=1024, restaurant=13312, this=18432, off-the-beaten=1024, Japanese=2048, food=2048, /=2048, accept=2048, the=2048, at=2048, reservations=2048, Are=2048, family-friendly=2048, Does=4096, location=2048, lunch?=2048, waitresses=1024, ?=6144}'),
 Row(word_count(json_0.text)='{told=1024, crowd=2048, allowed=1024, York,=1024, about=1024, whole=1024, sick=2048, during=1024, Day,=1024, muchClinton=1023, tried=1024, music=1024, would=1024, her=5120, Continued=1024, Story=1024, 11=1024, in=1024, ordinary=1024, is=1024, it=2048, being=1024, an=1024, I\'m=1024, former=1024, debuting=1024, at=3072, even=2048, circumstance

In [7]:
spark.stop()